In [1]:
import json
import re
import requests
from bs4 import BeautifulSoup
from os import path

In [2]:
def get_url(url):
    try:
        response = requests.get(url)
    except Exception as e:
        response = False
    
    if (response == False or response.status_code != 200):
        return ''
    
    return response.text

In [3]:
def get_dep_page_urls(container):
    li = container[0].find_all('li')
    
    dep_urls = []
    for group in li: 
        a = group.select('a')
        dep_urls.append(a[0]['href'])
    
    return dep_urls

In [4]:
def get_departments():
    url = 'https://www.uib.es/es/recerca/estructures/grups/grups_area/id_area=-1%2526npag={}'
    N = 16
    
    all_dep_urls = [];
    for i in range(1, N + 1):
        dep_url = url.format(i)
        text = get_url(dep_url)

        bs = BeautifulSoup(text, 'html.parser')

        container = bs.find_all("div", {"class": "uib_style_filaunica"})
        all_dep_urls.extend(get_dep_page_urls(container))
    
    return all_dep_urls


In [5]:
def get_research_group_url(department_url):
    text = get_url(department_url);
    
    bs = BeautifulSoup(text, 'html.parser')
    
    left_menu = bs.select('#menu-sticky')
    
    top_list = left_menu[0].select('div.titled_box')
    ul = top_list[1].select('ul.uib_style_menu_left')
    li = ul[0].find_all('li')
    a = li[0].find_all('a')
    
    group_url = a[0]['href']
    
    return 'https://www.uib.es{}'.format(group_url)

In [6]:
def get_group_memebers(group_url):
    text = get_url(group_url)
    
    bs = BeautifulSoup(text, 'html.parser')
    
    container = bs.select('#uib_style_content')
    if (len(container) == 0):
        return []
    
    ul = container[0].find_all('ul')
    
    all_people = []
    
    for content in ul:
        all_li = content.find_all('li')
        
        for person in all_li:
            link = person.find_all('a')
            
            if (len(link) == 0): # person without link
                all_people.append({
                    'name': person.text
                })
            else:  # has link
                 all_people.append({
                    'name': person.text,
                    'url': link[0]['href']
                })
    
    return all_people

In [7]:
# main progam
all_professors = []
all_deparments = get_departments()

In [8]:
for department in all_deparments:
    print(department)
    
    if "recerca/estructures/grups" not in department:
        continue
        
    group_url = get_research_group_url(department)
    members = get_group_memebers(group_url)
    
    all_professors.extend(members)

https://www.uib.es/es/recerca/estructures/grups/grup/GENEHUMA/
https://www.uib.es/es/recerca/estructures/grups/grup/BIOCANCER/
https://www.uib.es/es/recerca/estructures/grups/grup/BMC/
https://www.uib.es/es/recerca/estructures/grups/grup/EVES/
https://www.uib.es/es/recerca/estructures/grups/grup/GICAS/
https://www.uib.es/es/recerca/estructures/grups/grup/INFEINMU/
https://www.uib.es/es/recerca/estructures/grups/grup/ONCOHEMA/
https://www.uib.es/es/recerca/estructures/grups/grup/LITIRENA/
https://www.uib.es/es/recerca/estructures/grups/grup/GMEIN/
https://www.uib.es/es/recerca/estructures/grups/grup/NEUROCEL/
https://www.uib.es/es/recerca/estructures/grups/grup/FNEUROFA/
https://www.uib.es/es/recerca/estructures/grups/grup/FISIOLOG/
https://www.uib.es/es/recerca/estructures/grups/grup/NEUROSON/
https://www.uib.es/es/recerca/estructures/grups/grup/NUCOX/
https://www.uib.es/es/recerca/estructures/grups/grup/Global_Health/
https://www.uib.es/es/recerca/estructures/grups/grup/TERCIT/
https:

In [9]:
print(len(all_professors))
print(all_professors)

1510
[{'name': 'Dra. Cori Ramon Juanpere (Catedrática de universidad)', 'url': 'https://www.uib.cat/es/personal/ABDUwMjE/'}, {'name': 'Dr. José Aurelio Castro Ocón (Catedrático de universidad)', 'url': 'https://www.uib.cat/es/personal/ABDI3MTI/'}, {'name': 'Dra. Antonia Picornell Rigo (Catedrática de universidad)', 'url': 'https://www.uib.cat/es/personal/ABDE3MjI/'}, {'name': 'Sra. Marta Bassitta Sánchez (CAIB -> Contratada predoctoral)', 'url': 'https://www.uib.cat/es/personal/ABjIwODc3Mg/'}, {'name': 'Dr. José Antonio Jurado Rivera (Profesor contratado doctor interino)', 'url': 'https://www.uib.cat/es/personal/ABjIwNDQxMw/'}, {'name': 'Dra. Joana Francesca Ferragut Simonet (Profesora ayudante doctora)', 'url': 'https://www.uib.cat/es/personal/ABjIwNDE4NA/'}, {'name': 'Dra. Maria Magdalena Parera Rossello (Profesora asociada)'}, {'name': 'Sra. Beatriz Sánchez Barceló (Profesora asociada)', 'url': 'https://www.uib.cat/es/personal/ABjE4NDY5Nw/'}, {'name': 'Sra. Iris Alemany Hermoso (CAI